In [115]:
# ! pip install selenium
# ! pip install webcolors
# ! pip install --upgrade google-cloud-bigquery

# ! pip install validators

# ! apt-get update
! pip install chromium-chromedriver
! cp ./chromedriver /usr/bin

ERROR: Could not find a version that satisfies the requirement chromium-chromedriver (from versions: none)
ERROR: No matching distribution found for chromium-chromedriver
cp: ./chromedriver: No such file or directory


In [7]:
! pip install bigquery

     |████████████████████████████████| 171 kB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 19.1 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 8.2 MB/s  eta 0:00:01
     |████████████████████████████████| 142 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 514 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 206 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 380 kB 5.5 MB/s eta 0:00:01


  Created wheel for bigquery: filename=bigquery-0.0.20-py3-none-any.whl size=9161 sha256=b03f1a816abc050c4c522dd7a90f37b61b7b5b194316986328b2c8baf3605087
  Stored in directory: /Users/doguma/Library/Caches/pip/wheels/e1/99/21/a4b265bd46044ed58504686a85c2e787ba02cb7dfa7fb443e2
  Created wheel for dbstream: filename=dbstream-0.1.10-py3-none-any.whl size=7502 sha256=579d9854f90a3e556145aa72ab51e5ffbf8289b61f0d6b2817029fae3fe67de7
  Stored in directory: /Users/doguma/Library/Caches/pip/wheels/5b/5a/39/c988bd861bbcff0deec6086dd1f413f339c88d18e80c7d767e
  Created wheel for googleauthentication: filename=googleauthentication-0.0.16-py3-none-any.whl size=4735 sha256=d6bbfb8a8df57115cc18fab29b5678602722840950a882ccdb0455e6f589016a
  Stored in directory: /Users/doguma/Library/Caches/pip/wheels/8a/c3/b1/37190b8ddc140af443c19428a896c65e47f0f53aa32e2875a4
  Created wheel for dacktool: filename=dacktool-0.0.7-py3-none-any.whl size=4802 sha256=39f403bf70765ebaac5416ed76c8c3727bed97f70b35edb6a5f0450a1

In [4]:
import os, time, sys, io
import cv2, keras, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
from numpy import asarray
from io import BytesIO

from google.oauth2 import service_account
from google.cloud import bigquery

import requests
import PIL
import re
import urllib
import urllib.request
import webcolors
import validators
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image, ImageColor

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [116]:
def start_chromedriver():
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    
    return driver

In [18]:
# Hex to RGB Conversion
# Source: https://gist.github.com/kb22/f17e59a79d4fcca02188c23cca932be5#file-rgb2hex-py

def hex2name(c):
    h_color = '#{:02x}{:02x}{:02x}'.format(int(c[0]), int(c[1]), int(c[2]))

    rms_lst = []
    for img_clr, img_hex in webcolors.CSS3_NAMES_TO_HEX.items():
        cur_clr = webcolors.hex_to_rgb(img_hex)
        rmse = np.sqrt(mean_squared_error(c, cur_clr))
        rms_lst.append(rmse)

    closest_color = rms_lst.index(min(rms_lst))

    nm = list(webcolors.CSS3_NAMES_TO_HEX.items())[closest_color][0]

    return nm

In [8]:
# connecting to google cloud platform

! gcloud iam service-accounts create dopoem
! gcloud projects add-iam-policy-binding dopoem --member="serviceAccount:dopoem@dopoem.iam.gserviceaccount.com" --role="roles/owner"
! gcloud auth login

In [9]:
! export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads"
! export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/service-account-file.json"

In [7]:
# get json file from google cloud platform

credentials = "dopoem-f5aa6b994716.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials

In [8]:
# Google Cloud's client to pull data

client = bigquery.Client()

In [40]:
# Querying images and vision_api_data from "bigquery-public-data"

QUERY4 = ("""
SELECT * FROM (
  SELECT *
  FROM bigquery-public-data.the_met.images img
  LEFT JOIN (
    SELECT 
        *
    FROM bigquery-public-data.the_met.vision_api_data
    LEFT JOIN (
    SELECT 
        *
    FROM bigquery-public-data.the_met.objects
  )ON obj.object_id = vis.object_id
  ) vis
  ON img.object_id = vis.object_id
  WHERE obj.description = 'art'
)
LIMIT 3  # 500 for train, 200 for test
""")

In [155]:
# Querying the_met.objects data from "bigquery-public-data"

QUERY5 = ("""
SELECT object_id, LOWER(department), LOWER(object_name), \
        LOWER(culture), object_begin_date
        
FROM bigquery-public-data.the_met.objects,
UNNEST(SPLIT(culture, ',')) culture,
UNNEST(SPLIT(object_name, ',')) object_name,
UNNEST(SPLIT(department, ',')) department

LIMIT 400
""")

In [226]:
query_job5 = client.query(QUERY5)
rows5 = query_job5.result()

for i in rows5:
    print(i)

Row((38568, 'asian art', 'plaque', 'tibet', 1500), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2_': 3, 'object_begin_date': 4})
Row((41900, 'asian art', 'plaque', 'afghanistan', 1), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2_': 3, 'object_begin_date': 4})
Row((41900, 'asian art', 'plaque', ' possibly of west indian manufacture', 1), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2_': 3, 'object_begin_date': 4})
Row((60517, 'asian art', 'pipe case', 'japan', 1800), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2_': 3, 'object_begin_date': 4})
Row((60518, 'asian art', 'pipe case', 'japan', 1800), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2_': 3, 'object_begin_date': 4})
Row((38565, 'asian art', 'plaque', 'tibet (or nepal)', 1450), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2_': 3, 'object_begin_date': 4})
Row((38569, 'asian art', 'plaque', 'tibet', 1600), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2_': 3, 'object_begin_date': 4})
Row((38564, 'asian art', 'plaque', 'tibet', 1450), {'object_id': 0, 'f0_': 1, 'f1_': 2, 'f2

In [296]:
query_job5 = client.query(QUERY5)
rows5 = query_job5.result()

oid, o_type, o_country, o_date = [], [], [], []

for i in rows5:
    oid.append(i[0])
    o_type.append(i[2])
    o_country.append(i[3])
    o_date.append(i[4])

In [297]:
met_objects = pd.DataFrame()

met_objects['oid'], met_objects['o_type'], met_objects['o_country'], \
met_objects['o_date'] = oid, o_type, o_country, o_date

met_objects.head()

,oid,o_type,o_country,o_date
0,38568,plaque,tibet,1500
1,41900,plaque,afghanistan,1
2,41900,plaque,possibly of west indian manufacture,1
3,60517,pipe case,japan,1800
4,60518,pipe case,japan,1800


In [298]:
len(met_objects['o_type'].unique()), len(met_objects['o_country'].unique()),\
len(met_objects['o_date'].unique())


(71, 25, 32)

In [299]:
met_objects['o_country'].unique()

array(['tibet', 'afghanistan', ' possibly of west indian manufacture',
       'japan', 'tibet (or nepal)', 'burma', 'nepal', 'northwest china',
       'china', 'india', 'pakistan (ancient region of gandhara)',
       'india (jammu and kashmir', ' ancient kingdom of kashmir)', '',
       'sri lanka', 'india (orissa?)', 'india (orissa)',
       'india (andhra pradesh or west bengal)', 'india (karnataka',
       ' mysore or tamil nadu)', ' indo-portuguese (goa or diu)',
       'south india',
       'pakistan (ancient region of gandhara) or afghanistan',
       'india (bengal)', 'thailand'], dtype=object)

In [335]:
countries_crop = []

for i in met_objects['o_country']:
    temp = i
    temp = re.sub('\)', '', temp)
    
    if 'south india' in temp:
        temp = re.sub(' ', '-', temp)
        countries_crop.append(temp)
    elif 'india' in temp or 'indian' in temp:
        countries_crop.append('india')
    elif temp == '':
        countries_crop.append('others')
    elif temp.find('(')+1:
        temp = temp[:temp.find('(')].strip()
        temp = re.sub(' ', '-', temp)
        countries_crop.append(temp)
    else:
        temp = temp.strip()
        temp = re.sub(' ', '-', temp)
        countries_crop.append(temp)
        

In [336]:
set(countries_crop)

{'afghanistan',
 'ancient-kingdom-of-kashmir',
 'burma',
 'china',
 'india',
 'indo-portuguese',
 'japan',
 'mysore-or-tamil-nadu',
 'nepal',
 'northwest-china',
 'others',
 'pakistan',
 'south-india',
 'sri-lanka',
 'thailand',
 'tibet'}

In [337]:
met_objects['o_country'] = countries_crop

In [342]:
met_objects.dropna(inplace=True)
met_objects['o_country'].unique()

array(['tibet', 'afghanistan', 'india', 'japan', 'burma', 'nepal',
       'northwest-china', 'china', 'pakistan',
       'ancient-kingdom-of-kashmir', 'others', 'sri-lanka',
       'mysore-or-tamil-nadu', 'indo-portuguese', 'south-india',
       'thailand'], dtype=object)

In [306]:
len(met_objects)

400

In [340]:
! mkdir the_met_objects/train the_met_objects/test

for i in met_objects['o_country'].unique():
    ! mkdir the_met_objects/train/{i}
    
for i in met_objects['o_country'].unique():
    ! mkdir the_met_objects/test/{i}

In [125]:
def get_url(object_id):
    driver = start_chromedriver()
    driver.get('https://www.metmuseum.org/art/collection/search/' + str(object_id))
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')

    li_tag =  soup.find('a', 'gtm__download__image')
    href = li_tag['href']
    

    driver.quit()
    
    return href

In [101]:
def url_to_image(link):
  response = requests.get(link)
  # get image links that only respond with status code of 200
  # ignore the 'not found' pages
  if response.status_code == 200:
    image_bytes = io.BytesIO(response.content)
    img = PIL.Image.open(image_bytes)
    return img
  else:
    return None

In [ ]:
met_objects.tail()

In [343]:
remove_indexes = []

for index, el in met_objects[:320].iterrows():
    image_link = get_url(el['oid'])
    image = url_to_image(image_link)
    
    if image != None:
        image.save(r'the_met_objects/train/'+ str(el['o_country'])+\
                   '/' + str(el['oid'])+'.png')
    else:
        remove_indexes.append(j)

/Users/doguma/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (100920000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [341]:
remove_indexes2 = []

for index, el in met_objects[320:].iterrows():
    image_link = get_url(el['oid'])
    image = url_to_image(image_link)
    
    if image != None:
        image.save(r'the_met_objects/test/'+ str(el['o_country'])+\
                   '/' + str(el['oid'])+'.png')
    else:
        remove_indexes.append(j)

In [131]:
remove_indexes

[0, 1]

In [133]:
met_objects[2:].to_csv('met_objects')

---

In [47]:

met_images, met_id, type1, type2, type3, type4, color1, color2, color3, title, text = [], [], [], [], [], [], [], [], [], [], []

query_job4 = client.query(QUERY4)
rows4 = query_job4.result()




for i in rows4:

      image = url_to_image(i[3])
      if image != None:
        met_id.append(i[0])
#         img_numpy = asarray(image)
#         met_images.append(img_numpy)
        image.save(r'met_images_obj/s'+str(i[0])+'.png')

        # driver = start_chromedriver()
        # driver.get('https://www.metmuseum.org/art/collection/search/' + str(i[0]))
        # html = driver.page_source

        # soup = BeautifulSoup(html, 'html.parser')
  
        # title.append(soup.find('span', 'artwork__title__inset').text)
        # text.append(soup.find('div', 'artwork__intro__desc').text)

        # driver.quit()

        # try and except due to missing values
        try:
          type1.append(i[12][0]['description'])
        except:
          type1.append(None)
        try:
          type2.append(i[12][1]['description'])
        except:
          type2.append(None)
        try:
          type3.append(i[12][2]['description'])
        except:
          type3.append(None)
        try:
          type4.append(i[12][3]['description'])
        except:
          type4.append(None)

        # I should use functions more next time..
        color1.append(hex2name((i[15]['dominantColors']['colors'][0]['color']['blue'],\
                i[15]['dominantColors']['colors'][0]['color']['green'],\
                i[15]['dominantColors']['colors'][0]['color']['red'])))

        color2.append(hex2name((i[15]['dominantColors']['colors'][1]['color']['blue'],\
                i[15]['dominantColors']['colors'][1]['color']['green'],\
                i[15]['dominantColors']['colors'][1]['color']['red'])))
        
        color3.append(hex2name((i[15]['dominantColors']['colors'][2]['color']['blue'],\
                i[15]['dominantColors']['colors'][2]['color']['green'],\
                i[15]['dominantColors']['colors'][2]['color']['red'])))

In [36]:
met_collec_data = None
met_collec_data = pd.DataFrame()
met_collec_data['id'] = met_id
met_collec_data['type1'] = type1
met_collec_data['type2'] = type2
met_collec_data['type3'] = type3
met_collec_data['type4'] = type4
met_collec_data['color1'] = color1
met_collec_data['color2'] = color2
met_collec_data['color3'] = color3

In [108]:
import pickle

In [109]:
filename = 'met_total_data'

outfile = open(filename, 'wb')

pickle.dump(met_collec_data, outfile)
outfile.close()